In [99]:
import os
import json
import random

root_dir = '../MATH'  # Replace with your directory
num_files_to_load = 1000  # Replace with number of files you want to pick

# Set a seed for randomness
random.seed(123)

json_files = []

# Search for json files in sub-folders of root directory
for dirpath, _, filenames in os.walk(root_dir):
    for f in filenames:
        if f.endswith('.json'):
            json_files.append(os.path.join(dirpath, f))

# Randomly select files
files_to_load = random.sample(json_files, num_files_to_load)

data = []

# Load content from files
for filename in files_to_load:
    with open(filename, 'r') as f:
        data.append(json.load(f))

# print(data)

In [100]:
extracted_data = []
converted_data = []

for datum in data:
    problem = datum.get('problem')
    solution = datum.get('solution')
    # Extract the boxed solution
    boxed_solution = ""
    if "\\boxed{" in solution:
        start = solution.index("\\boxed{") + len("$\boxed{")
        end = solution.index("}", start)
        boxed_solution = solution[start:end]
    extracted_data.append({'problem': problem, 'boxed_solution': boxed_solution})
    
# Check if only numbers and periods in boxed_solution, convert to float
for item in extracted_data:
    boxed_solution = item['boxed_solution']
    if all(c.isdigit() or c == '.' for c in boxed_solution):
        item['boxed_solution'] = float(boxed_solution)
        converted_data.append(item)

# print(converted_data)

In [101]:
with open('sanitized_data.jsonl', 'w') as outfile:
    for entry in converted_data:
        json.dump(entry, outfile)
        outfile.write('\n')